In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

from tqdm import tqdm

import random
from PIL import Image

import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def setseed(seednum = 20):
    torch.manual_seed(seednum)
    torch.cuda.manual_seed(seednum)
    torch.cuda.manual_seed_all(seednum)
    np.random.seed(seednum)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(seednum)

setseed(35)

In [4]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

device

device(type='cuda')

In [5]:
transform  = transforms.Compose([transforms.ToTensor(),
           transforms.Normalize((0.5,0.5, 0.5), (0.5, 0.5, 0.5))])


aug = transforms.Compose([transforms.ToTensor(),
                          transforms.RandomHorizontalFlip(p=0.5),
                          transforms.RandomGrayscale(p=0.5)
          ])





In [7]:
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/Statistical_Deep_Image",
                                           transform = transform)

In [6]:
trainset1 = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=True,
                                        transform=transform)

trainset2 = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=False,
                                        transform=aug)



vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=True,
                                        transform=transform)

trainset = trainset1+ trainset2


Files already downloaded and verified
Files already downloaded and verified


In [9]:
train_loader = DataLoader(trainset,
                          batch_size = 64,
                          shuffle=True)

vaild_loader = DataLoader(vaildset,
                          batch_size = 64,
                          shuffle=True)


test_loader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle=False)


In [28]:
class IdentityPadding(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(IdentityPadding, self).__init__()

        if stride == 2:
            self.pooling = nn.AvgPool2d(kernel_size=2, stride=2, ceil_mode=True)
        else:
            self.pooling = None
            
        self.add_channels = out_channels - in_channels
    
    def forward(self, x):
        out = F.pad(x, (0, 0, 0, 0, 0, self.add_channels))
        if self.pooling is not None:
            out = self.pooling(out)
        return out


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                                stride=stride, padding=1, bias=False)      
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, 
                                stride=1, padding=1, bias=False)    
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.down_sample = IdentityPadding(in_channels, out_channels, stride)
            
        self.stride = stride

    def forward(self, x):
        shortcut = self.down_sample(x)
        out = self.bn1(x)
        out = self.conv1(out)        
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn3(out)
       
        out += shortcut
        return out


class PyramidNet(nn.Module):
    def __init__(self, num_layers, alpha, block, num_classes=10):
        super(PyramidNet, self).__init__()   	
        self.in_channels = 16
        
        # num_layers = (110 - 2)/6 = 18
        self.num_layers = num_layers
        self.addrate = alpha / (3*self.num_layers*1.0)

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, 
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)

        # feature map size = 32x32
        self.layer1 = self.get_layers(block, stride=1)
        # feature map size = 16x16
        self.layer2 = self.get_layers(block, stride=2)
        # feature map size = 8x8
        self.layer3 = self.get_layers(block, stride=2)

        self.out_channels = int(round(self.out_channels))
        self.bn_out= nn.BatchNorm2d(self.out_channels)
        self.relu_out = nn.ReLU(inplace=True)
        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.fc_out = nn.Linear(self.out_channels, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', 
                                        nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def get_layers(self, block, stride):
        layers_list = []
        for _ in range(self.num_layers - 1):
            self.out_channels = self.in_channels + self.addrate
            layers_list.append(block(int(round(self.in_channels)), 
                                     int(round(self.out_channels)), 
                                     stride))
            self.in_channels = self.out_channels
            stride=1

        return nn.Sequential(*layers_list)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.bn_out(x)
        x = self.relu_out(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc_out(x)
        return x


def pyramidnet():
	block = ResidualBlock
	model = PyramidNet(num_layers=18, alpha=48, block=block)
	return model

In [29]:
net = pyramidnet().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

total=0 
correct = 0

In [ ]:

optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [30]:
for epoch in range(20):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0
    if i %1500 ==1499:
      with torch.no_grad():
        val_loss = 0.0
        for k, data1 in enumerate(vaild_loader, 0):
          val_inputs, val_label = data1[0].to(device), data1[1].to(device)
          val_output = net(val_inputs)
          v_loss = criterion(val_output, val_label)
          val_loss += v_loss
          _, predicted = torch.max(val_output.data,1)
          total += val_label.size(0)
          correct += (predicted == val_label).sum().item()
      print("validation loss {}".format(val_loss))
      print("vaildset Accuracy  : {}".format(100* correct/total))
      total=0
      correct=0
      with torch.no_grad():
        for data in test_loader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs= net(images)
          _, predicted = torch.max(outputs.data,1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print("testset Accuracy  : {}".format(100* correct/total))
      total=0
      correct=0


Epoch: 1,Batch : 750, Loss:0.5358088705837727
Epoch: 1,Batch : 1500, Loss:0.36079653626680375
validation loss 135.14242553710938
vaildset Accuracy  : 69.19
testset Accuracy  : 13.1
Epoch: 2,Batch : 750, Loss:0.2828348472714424
Epoch: 2,Batch : 1500, Loss:0.2417343026995659
validation loss 95.97380065917969
vaildset Accuracy  : 79.06
testset Accuracy  : 15.85
Epoch: 3,Batch : 750, Loss:0.20689992441236973
Epoch: 3,Batch : 1500, Loss:0.19019811673462392
validation loss 82.16142272949219
vaildset Accuracy  : 81.86
testset Accuracy  : 17.6
Epoch: 4,Batch : 750, Loss:0.16490997467190027
Epoch: 4,Batch : 1500, Loss:0.15965054617077112
validation loss 71.69403076171875
vaildset Accuracy  : 84.51
testset Accuracy  : 17.95


TypeError: ignored

pretrained 넣기


In [21]:
PATH = '/content/drive/MyDrive/share/trained_model/PyramidNet_Aug_weights.pth'



In [22]:
torch.save(net.state_dict(), PATH)

In [15]:


net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [25]:
correct = 0
total = 0
with torch.no_grad():
  for data in train_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("trainset Accuracy  : {}".format(100* correct/total))

trainset Accuracy  : 10.667


In [26]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("vaildset Accuracy  : {}".format(100* correct/total))

vaildset Accuracy  : 10.49


In [27]:
correct = 0
total = 0

with torch.no_grad():
    for data in tqdm(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy: %d %%' % (100 * correct / total))
print(correct)
print(total)

100%|██████████| 32/32 [00:04<00:00,  7.45it/s]

Accuracy: 10 %
202
2000
